Atomistic simulations are becoming more and more accessible thanks to the development of easy-to-use frameworks that deal with the setup, running and analysis stages.

The Atomistic Simulation Environment ([ASE](https://wiki.fysik.dtu.dk/ase)) is one such framework that has interfaces to many different codes as well as a large suite of tools to aid in creation of the system and analysis of its properties.

This notebook will contain an ongoing exploration of the ASE's features as I mock up various structures as a sort of playful way to learn how to use it. 

If you have any suggestions for a system that would be interesting to create, or how to improve any of the systems you see here (they are very rough), please get in touch!

In [1]:
# Generally useful imports
from ase.visualize import view
import numpy as np

## Nanotubes

In [2]:
from ase.build import nanotube
from ase.constraints import FixAtoms

### Basic carbon nanotube

In [59]:
cnt = nanotube(3,3, length=8)
cnt.write("images/CNT_example.pov",
         rotation="4.75x,-5.4y,88.1z",
         run_povray=True)
# view(cnt)

![Example carbon nanotube](images/CNT_example.png)

### Multi-walled boron nitride nanotube

Let's say you wanted to put a boron nitride (BN) nanotube inside a bigger nanotube (because science).
There's now a degree of freedom in the relative rotation of the outer and inner nanotubes along the tube axis.

Something like this:
![BN nanotube schematic](images/rot_nt_diagram.png)

We can explore this degree of freedom manually by rotating the inner tube, or use an ASE geometry optimiser together with some clever constraint. The following loop sets up the former:

In [46]:
writeimages = False
tubes = []
for thetadeg in range(0, 360, 2):
    theta = thetadeg * np.pi / 180.0
    cnt1 = nanotube(3,3,length=5, bond= 1.48, symbol='B')
    # rotate inner cnt by theta about z, centered in the center of the unit cell
    cnt1.rotate('z',theta, center='COU')
    cnt2 = nanotube(6,6,length=5, bond= 1.48, symbol='N')
    # now replace every second atom with B
    for i, atom in enumerate(cnt1):
        if i%2==0:
            cnt1[i].symbol = 'N'
    for i, atom in enumerate(cnt2):
        if i%2==0:
            cnt2[i].symbol = 'B'

    # fix all outer nt atoms in place
    mask = [True for atom in cnt2]
    cnt2.set_constraint(FixAtoms(mask=mask))
    # let inner nt relax
    mask1 = [False for atom in cnt1]
    cnt1.set_constraint(FixAtoms(mask=mask1))

    # now set them to the same cell and centre
    middleofcnt1 = cnt1.get_cell_lengths_and_angles()[0:3] /2
    cnt1.set_cell(cnt2.get_cell())
    cnt1.center(about=middleofcnt1)

    atoms = cnt2.copy()
    atoms.extend(cnt1)

    # add vacuum of 10A
    atoms.center(vacuum=10, axis=(0,1))
    tubes.append(atoms)
    if writeimages:
    #     atoms.write("images/rot_nt_{0:03d}.png".format(thetadeg), rotation="1.72x,-5.72y")
        atoms.write("images/rot_nt_{0:03d}.pov".format(thetadeg),
                    rotation="1.72x,-5.72y", # get this angle from ase-gui 
                    run_povray=True)


We can use the ase-gui to view the resulting tubes.

In [38]:
view(tubes)

If we write out the images to png, we can animate the files using [imagemagik](https://www.imagemagick.org/script/index.php)

In [47]:
!convert -loop 0 -delay 5 images/rot_nt_???.png images/nt_BN_rot.gif

Here's the result (fancy eh?):
![Animated loop of BN nanotubes rotating](images/nt_BN_rot.gif)

There's a sort optical illusion that the outer tube is rotating also; or have I just been staring at code for too long?

`TODO` add clever constraint and how to use the ASE optimiser 

### Bundle of nanotubes

Let's say you wanted to form a bundle of nanotubes (they must form bundles... right?). 

The way the bundle bundles up is similar to stacking canonballs, but in 2D. I mean, looking down the tube axis you'd see something like one of these options:
![lattice of nts](images/nt_bundle_schematic.png)

Clearly the one on the right is a better way to bundle the tubes, but how do we define the edges of the bundle? Since this is a hexagonal bundling pattern, it makes sense to have a hexagonal bundle. Something like this: 

![Hexagonal lattice](images/hex_bundle.png)

I'm sure there's a clever way of getting that nice bundle of nanotubes, but for now let's just hack something together:

In [18]:
# These parameters need to be adjusted to get right!
#cnt diameter
d = 4.25/2
# separation - assume bond length-ish
s = 1.45*1.1
s2 = 1.45*2

cnt1 = nanotube(3,3,length=4, vacuum=30)
bundle = cnt1.copy()
# Repeat tubes along x
for i in range(1,7):
    # Repeat tubes along y
    for j in range(0,4):
        # Hack to cut out a hexagon - dumb but works
        if j >1 and i < 2:
            pass
        elif j>2 and i < 3:
            pass
        else:
            # Treat the first row specially to avoid duplicates
            if j == 0:
                cnt2 = cnt1.copy()
                cnt2.rotate('z',np.pi/6., center='COU')
                cnt2.translate((i*(d+s2), 0, 0))
                bundle.extend(cnt2)
            else:
                cnt3 = cnt1.copy()
                cnt4 = cnt1.copy()

                cnt3.rotate('z',np.pi/6., center='COU')
                cnt3.translate((i*(d+s2)-j*((d+s2)/2),j*(d+s),0))
                bundle.extend(cnt3)

                cnt4.rotate('z',np.pi/6., center='COU')
                cnt4.translate((i*(d+s2)-j*((d+s2)/2),-j*(d+s),0))
                bundle.extend(cnt4)
    
    
bundle.center()
view(bundle)
bundle = bundle*(1,1,10)

Saving an image of the bundle:

In [20]:
bundle.write("images/cnt_hex_bundle_test.pov",
             rotation="1.72x,-2.7y", # get this angle from ase-gui 
             show_unit_cell=2,
             run_povray=True)
bundle.write("images/cnt_hex_bundle_2.png",
             rotation="18x,90z", # get this angle from ase-gui 
             show_unit_cell=2)
#              run_povray=True)

And we end up with something like what we wanted:

![Bundle of CNT's](images/cnt_hex_bundle_test.png)
![Bundle of CNT's 2](images/cnt_hex_bundle_2.png?que)

`NB` the tube diameter, relative rotations and separation parameters all need to be adjusted to really give a sensible initial state.

## Nanoribbons

Nanoribbons (NR) are what you'd get if you slice ribbons out of single-atom layer of something (e.g. graphene). 
ASE has a generator to help make NRs.

The two basic types are armchair and zigzag - like with nanotubes.

In [3]:
from ase.build import graphene_nanoribbon

In [4]:
# These are mostly the default parameters
gnr_ac = graphene_nanoribbon(5, 5,
                        type='armchair',
                        saturated=False,
                        C_H=1.09,
                        C_C=1.42,
                        vacuum=5,
                        magnetic=None, initial_mag=1.12,
                        sheet=False,
                        main_element='C', saturate_element='H')
# also default settings, just saving space:
gnr_zz = graphene_nanoribbon(5, 5,
                             type='zigzag',
                             vacuum=5)
#The ribbons extend in z - let's emphasise that here:
gnr_ac = gnr_ac*(1,1,2)
#zz has a smaller repeating unit
gnr_zz = gnr_zz*(1,1,4)

# view(gnr_ac)

gnr_ac.write("images/graphene_nr_armchair.pov",
             rotation="90x,90z",
             run_povray=True,
             show_unit_cell=2)
gnr_zz.write("images/graphene_nr_zigzag.pov",
             rotation="90x,90z",
             run_povray=True,
             show_unit_cell=2)

We get the armchair:
![Armchair graphene nanoribbon](images/graphene_nr_armchair.png)
and the zigzag:
![Zigzag graphene nanoribbon](images/graphene_nr_zigzag.png?3)

That's cool, but what else can we do with ribbons? In the Olympics ribbons are waved around a lot - let's see if we can make our ribbons wave.

The ribbon extends in the z direction, and has some width in the x direction. That leaves us the y direction to wave. Now because of the periodic boundary conditions, only certain wavelengths are allowed - if you had for example 1.8 wavelengths in the box, the beginning and end points would join up!

For now let's just wave the ribbon in the y direction, ignoring the fact that we're stretching the bonds a little.

In [13]:
# box limit in z:
maxz = gnr_zz.get_cell_lengths_and_angles()[2]
# Amplitude of wave:
a = 2
# mode
n = 2
atoms = gnr_zz.copy()
for atom in atoms:
    atom.position[1] =  a * np.sin(2*n*np.pi*atom.position[2]/maxz) +  atom.position[1]
# view(atoms)
atoms.write("images/nr_zigzag_wave-1.pov",
             rotation="90x,90z",
             run_povray=True,
             show_unit_cell=2)
atoms.write("images/nr_zigzag_wave-2.pov",
             rotation="90y",
             run_povray=True,
             show_unit_cell=2)
atoms.write("images/nr_zigzag_wave-3.pov",
             rotation="67x,-72y,-62z",
             run_povray=True,
             show_unit_cell=2)

This then is our wavy nanoribbon, seen from different angles:

![Wavy nanoribbon 1](images/nr_zigzag_wave-1.png?2)
![Wavy nanoribbon 2](images/nr_zigzag_wave-2.png?2)
![Wavy nanoribbon 3](images/nr_zigzag_wave-3.png?2)